# Use functions for everything

In [1]:
from requests_html import HTMLSession
import pandas as pd
from tqdm import tqdm_notebook, tqdm
%matplotlib inline

url = 'http://apf.ststephens.edu/attendance_stku2018'
get_semester_url = 'http://apf.ststephens.edu/getsemesterbypro?programme_id={}'
get_month_url = 'http://apf.ststephens.edu/getmonthbysemepro?semester_id={}&programme_id={}'

In [17]:

def scrape_attendance():
   
    session = HTMLSession()
    r = session.get(url) 
    course_options = r.html.find('option')[1:33]
    session.close()
    programs = {  course.attrs['value'] : course.text
            for course in course_options }
    all_attendance = []
    for program in tqdm_notebook(programs, desc='Program'):
        df = scrape_course(program, programs)
        all_attendance.append(df)
    attendance_df = pd.concat(all_attendance)
    return tidy_scraped(attendance_df)
        
    
def scrape_course(programme_id,programs_dict):
    session = HTMLSession()
    p = session.get(get_semester_url.format(programme_id))
    sem_dict = p.json()
    
    if sem_dict['status']:
        semesters = {}
        for entry in sem_dict['semester']:
            semesters[entry['semester_id']] = entry['name']
    semesters_attendance = []
    for semester in tqdm_notebook(semesters,desc = 'Semester', leave=False):
        try:
            semesters_attendance.append(
                scrape_semester(
                    programme_id=programme_id,
                    semester_id=semester,
                    semesters_dict=semesters,
                    programs_dict=programs_dict,
                    html_session = session
                )
            )
        except ValueError:
            pass
    total_attendance = pd.concat(semesters_attendance)
    session.close()
    return total_attendance

    
def scrape_semester(programme_id, semester_id, semesters_dict, programs_dict, html_session):
    
    q = html_session.get(get_month_url.format(semester_id,programme_id))
    month_dict = q.json()
    
    if month_dict['status'] and month_dict['month'] != []:
        months = {}
        for entry in month_dict['month']:
            months[entry['month_no']] = entry['name']   
    months_attendance = []
    for month in tqdm_notebook(months,desc='Month', leave=False):
        try:
            months_attendance.append(
                scrape_month(
                    programme_id=programme_id,
                    semester_id=semester_id, 
                    month_id=month, 
                    months_dict=months, 
                    semesters_dict=semesters_dict,
                    programs_dict = programs_dict,
                    html_session = html_session
                )
            )
        except ValueError:
            pass
    semester_attendance = pd.concat(months_attendance)
    return semester_attendance

def scrape_month(programme_id, semester_id, month_id, months_dict, semesters_dict,programs_dict,html_session):
    payload = {
        'programme' : programme_id,
        'semester' : semester_id,
        'month' : month_id
    }
    
    s = html_session.post(url, data=payload)
    if s.status_code == 200: # A OK
        try:
            scraped_table = pd.read_html(s.html.html, header=1)[0] # Read the table into Pandas
        except ValueError:
            raise
        # Perform some formatting to get it into order
        scraped_table.drop(columns=scraped_table.columns.values.tolist()[-11:], inplace=True)
        scraped_table['Month'] = months_dict[payload['month']]
        scraped_table['Semester'] = semesters_dict[payload['semester']]
        scraped_table['Year'] = programs_dict[payload['programme']].split(' ')[0]
        scraped_table['Course'] = ' '.join(programs_dict[payload['programme']]
                                           .split(' ')[2:]
                                          ).replace('B. A. HONOURS ','').replace('B. SC. HONOURS ','')
        # Identify papers and name columns accordingly
        papers = [
            paper.text 
            for paper in s.html.find(selector='th') 
            if 'colspan' in paper.attrs and paper.attrs['colspan'] == '6'
        ]
        types = ['Lectures_Held', 'Lectures_Attended',
                 'Tutorials_Held','Tutorials_Attended',
                 'Practicals_Held', 'Practicals_Attended',]
        headers = ['Name']
        for paper in papers:
            for attendance_type in types:
                headers.append(paper+'_'+attendance_type)     
        headers.extend(['Month', 'Semester', 'Year', 'Course'])
        col_names = scraped_table.columns.values.tolist()
        col_rename = dict(zip(col_names,headers))
        scraped_table.rename(mapper=col_rename, axis=1,inplace=True)
        return scraped_table
    else:
        return None
    

def tidy_scraped(scraped_table):
    scraped_melted = pd.melt(scraped_table,id_vars=['Name', 'Month', 'Semester','Year','Course'], var_name='Paper_Attendance')
    paper_types = scraped_melted['Paper_Attendance'].str.split('_')
    paper_names = paper_types.str.get(0)
    class_type = paper_types.str.get(1)
    attendance_type = paper_types.str.get(2)
    scraped_melted['Paper Name'] = paper_names
    scraped_melted['Class Type'] = class_type
    scraped_melted['Attendance Type'] = attendance_type
    scraped_melted.drop('Paper_Attendance', axis =1, inplace=True)
    attend_tidy = scraped_melted.pivot_table(
        index=['Year','Course','Name', 'Semester','Paper Name', 'Month', 'Class Type'],
        columns='Attendance Type',
        values='value'
    )
    attend_tidy['Percent'] = (attend_tidy['Attended']/attend_tidy['Held']*100).round(2)
    attend_tidy.dropna(axis=0, how='any', inplace=True)
    return attend_tidy

In [18]:
df = scrape_attendance()

HBox(children=(IntProgress(value=0, description='Program', max=32), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Semester', max=2), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Month', max=5), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Month', max=5), HTML(value='')))

KeyboardInterrupt: 

In [4]:
df.to_pickle('scraped_data.pkl')

In [6]:
df.head(50)

Attendance Type                                                                                                      Attended  \
Year Course          Name         Semester     Paper Name                                      Month     Class Type             
I    B. A. PROGRAMME Aadil Gulzar Semester I   Core English                                    August    Lectures         9.0   
                                                                                                         Tutorials        0.0   
                                                                                               July      Lectures         4.0   
                                                                                               November  Lectures         7.0   
                                                                                                         Tutorials        0.0   
                                                                                               October   Lectures         7.0   
                                                                                                         Tutorials        2.0   
                                                                                               September Lectures        11.0   
                                                                                                         Tutorials        1.0   
                                               Introduction to Political Theory                August    Lectures         8.0   
                                                                                               July      Lectures         8.0   
                                                                                               November  Lectures        14.0   
                                                                                                         Tutorials        0.0   
                                                                                               October   Lectures         8.0   
                                                                                                         Tutorials        1.0   
                                                                                               September Lectures         8.0   
                                               Principles of Microeconomics - I                August    Lectures         5.0   
                                                                                                         Tutorials        0.0   
                                                                                               July      Lectures         1.0   
                                                                                               November  Lectures         4.0   
                                                                                                         Tutorials        1.0   
                                                                                               October   Lectures         9.0   
                                                                                                         Tutorials        2.0   
                                                                                               September Lectures         9.0   
                                                                                                         Tutorials        1.0   
                                               Urdu B AECC                                     August    Lectures        16.0   
                                                                                                         Tutorials        2.0   
                                                                                               July      Lectures         6.0   
                                                                                                         Tutorials        1.0   
 

In [8]:
df2= pd.read_pickle('scraped_data.pkl')

In [9]:
df2

Attendance Type                                                                                                      Attended  \
Year Course          Name            Semester     Paper Name                                   Month     Class Type             
I    B. A. PROGRAMME Aadil Gulzar    Semester I   Core English                                 August    Lectures         9.0   
                                                                                                         Tutorials        0.0   
                                                                                               July      Lectures         4.0   
                                                                                               November  Lectures         7.0   
                                                                                                         Tutorials        0.0   
                                                                                               October   Lectures         7.0   
                                                                                                         Tutorials        2.0   
                                                                                               September Lectures        11.0   
                                                                                                         Tutorials        1.0   
                                                  Introduction to Political Theory             August    Lectures         8.0   
                                                                                               July      Lectures         8.0   
                                                                                               November  Lectures        14.0   
                                                                                                         Tutorials        0.0   
                                                                                               October   Lectures         8.0   
                                                                                                         Tutorials        1.0   
                                                                                               September Lectures         8.0   
                                                  Principles of Microeconomics - I             August    Lectures         5.0   
                                                                                                         Tutorials        0.0   
                                                                                               July      Lectures         1.0   
                                                                                               November  Lectures         4.0   
                                                                                                         Tutorials        1.0   
                                                                                               October   Lectures         9.0   
                                                                                                         Tutorials        2.0   
                                                                                               September Lectures         9.0   
                                                                                                         Tutorials        1.0   
                                                  Urdu B AECC                                  August    Lectures        16.0   
                                                                                                         Tutorials        2.0   
                                                                                               July      Lectures         6.0   
                                                                                                         Tutorials        1.0   
 

In [16]:
df2.groupby(['Year']).mean()

Attendance Type,Attended,Held,Percent
Year,,,
I,7.340337,10.070611,inf


In [13]:
df2.loc

In [51]:
for program in tqdm_notebook(test, desc='Program'):
    df = scrape_course(program, programs)
    df.to_csv('./data/raw/{}.csv'.format(programs[program]))
    df2 = tidy_scraped(df)
    df2.to_csv('./data/tidy/{}.csv'.format(programs[program]))

HBox(children=(IntProgress(value=0, description='Program', max=2), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Semester', max=2), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Month', max=5), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Month', max=5), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Semester', max=2), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Month', max=5), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Month', max=5), HTML(value='')))

In [53]:
names = index.get_level_values(0).unique().tolist()

In [95]:
att = df2.loc['Ojaswini Shekhawat']
att.groupby('Month').mean()['Percent']

Month
August        86.183750
February      87.550000
January       79.488333
July         100.000000
November       7.575000
October       57.576250
September     65.830000
Name: Percent, dtype: float64

In [69]:
att.groupby('Month').mean()['Percent']

Month
August        86.183750
February      87.550000
January       79.488333
July         100.000000
November       7.575000
October       57.576250
September     65.830000
Name: Percent, dtype: float64